# 🚀 Projet TortillaPredict !

**Ce projet aura pour objectif d'étudier d'étudier des données relatives aux prix de vente de tortillas, au mexique, en fonction de différentes caractéristiques.**

Nous procéderons dans un premier temps à une analyse des données afin de mieux les appréhender, puis nous tenterons d'établir un modèle de prédiction permettant de prédire le prix auquel sera vendue une tortilla en fonction de ses caractéristiques.